# Chapter 6 - Managed Agents: Customer Support Agent with Knowledge Bases and Action Groups

## Overview
This notebook demonstrates how to build a comprehensive customer support agent using Amazon Bedrock Agents. We'll create an agent that can handle order management, product inquiries, and customer service tasks by integrating with knowledge bases and backend systems through action groups.

In this notebook, we will build a customer support agent using Agents and Knowledge bases in Amazon Bedrock. We will leverage Anthropic Claude 3 foundational models from Amazon Bedrock using the Boto3 SDK.

<img src="images/agent-architecture.png" style="width:70%;display:block;margin: 0 auto;">
<br/>

On a high-level, we will cover 3 key steps : 

#### Setup 
- Import the needed libraries and utilities.

#### Agents 
 - Create the Agent
 - Create the Action Groups 
 - Create Lambda functions to call external APIs/Systems and associate them with Actions Groups
 - Associate the knowledge base with agent to improve the relevance and accuracy of the agent responses
 - Test the agent.

#### Clean-up 
 - Remove any resources created

In our exampmle for this book, Customer Support Agent can perform following actions/tasks -
- Place a new order
- Cancel the order
- Retrieve the order details and tracking information
- Retrive product details from knowledge base

## 1. Setup - import the required libraries

First step is to install the pre-requisites packages

In [1]:
%pip install --upgrade -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import constants
from constants import AGENT_FOUNDATION_MODEL, CUSTOMER_SUPPORT_KB_ID, ORDER_LAMBDA_CODE_FILE_NAME


In [2]:
import boto3
import json
import time
import logging
import pprint
import random

In [ ]:
# Initialize logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Note: AWS clients are initialized in the utility functions

In [ ]:
# Import helper functions and AWS clients from utility module
from agents_helper_util import (
    create_agent_role_and_policies, 
    create_lambda_role, 
    delete_agent_roles_and_policies,
    create_dynamodb, 
    create_lambda, 
    clean_up_resources,
    bedrock_agent_client,
    bedrock_agent_runtime_client,
    lambda_client,
    account_id,
    region
)

In [ ]:
# Display current AWS configuration
print(f"Region: {region}")
print(f"Account ID: {account_id}")

In [7]:
# Create a DynamoDB table for order tracking
table_name = 'orders'
partition_key = 'order_id'
tbl = create_dynamodb(table_name, partition_key)

In [ ]:
# Agent parameters
random_digits = ''.join(random.choices('0123456789', k=2))
agent_name = f"cust-support-agent-z{random_digits}"
# agent_name = 'cust-support-agent-api-2'
agent_foundation_model = AGENT_FOUNDATION_MODEL
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_alias_name = "customer-support-agent-dev"
agent_description = "You are a friendly, helpful, polite and knowledgeable customer support agent. Your goal is to assist customers with questions related to order, and products."
order_agent_action_group_name = "order-action-group"
order_agent_action_group_description = """Place order, get order details, tracking and cancel action group."""

In [ ]:
# Agent Instructions are required and will be used by agent to define its purpose.
agent_instruction = """
I want you to act as a friendly, helpful, and knowledgeable customer support agent. Your goal is to assist customers with inquiries related to creating new orders, retrieving order details or tracking orders, cancelling orders, and providing product information in a clear and succinct manner. You should not share your rationale or thinking directly with customer and prepare a friendly response as per below guidelines.
1) Use natural, conversational language that a real customer service representative would use.
2) Ask for any necessary information from the customer in a polite and helpful way, without implying or stating any system constraints.
3) Never reveal or reference any internal implementation details, functions, systems, parameters, knowledge bases, requirements for exact product names, or any other technical details about how the system works behind the scenes. 
4) If you don't have enough information to fully address the customer's request, politely ask for clarification or additional details needed, just as a human agent would.
5) Always prioritize a positive, supportive experience for the customer over technical accuracy or completeness.
6) Never mention, describe or hint at your own internal processes, limitations or mechanisms for processing customer requests.
For example:
Example 1: If a customer asks "Can you help me place or create a new order?", respond: "Certainly! I'd be happy to assist you with placing a new order. To get started, could you please provide me with your full name, product you want to buy, shipping address where you'd like the order delivered and your preferred payment method? Once I have those details, I can help order the product you need."
Example 2: If a customer asks to cancel an order, respond: "I'm sorry you need to cancel your order. Could you please provide the order number so I can locate and process the cancellation?"
Example 3: If a customer asks to track an order, respond: "No problem, I'd be happy to look up the status of your order. Could you please provide your order number?"
"""

# Knowledge base parameters : Get the knowledge base id you created in chapter 5. 
customer_support_kb_id = CUSTOMER_SUPPORT_KB_ID # Replace with your knowledge base id.
customer_support_kb_version = "1"


In [ ]:
# Create agent role and policies
agent_role = create_agent_role_and_policies(agent_name, agent_foundation_model, kb_id=customer_support_kb_id)
agent_role

In [ ]:
# Create the customer support agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

In [ ]:
customer_support_agent_id = response['agent']['agentId']
print("The agent id is:",customer_support_agent_id)

In [13]:
# # Create the Order and ReturnRefund Action Groups
order_action_group_agent_functions = [
    {
        'name': 'place-order',
        'description': 'Use this function to place or crate an order',
        'parameters': {
            "product_name": {
                "description": "Product Name",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Customer Name",
                "required": True,
                "type": "string"
            },
            "quantity": {
                "description": "Amount of product being ordered. ",
                "required": True,
                "type": "string"
            },
            "shipping_address": {
                "description": "Shipping Address",
                "required": True,
                "type": "string"
            },
            "payment_method": {
                "description": "Payment Method",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'retrieve-order-tracking-info',
        'description': 'Use this  function to retrieve and track order details',
        'parameters': {
            "order_id": {
                "description": "Order Id",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'cancel-order',
        'description': 'Use this function to cancel the order.',
        'parameters': {
            "order_id": {
                "description": "Order Id",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [ ]:
# Create the Lambda Functions for the Order Action Groups
lambda_iam_role = create_lambda_role(agent_name)
lambda_code_file_name = ORDER_LAMBDA_CODE_FILE_NAME

order_lambda_function_name = f"order-{agent_name}"
order_lambda_function = create_lambda(order_lambda_function_name,lambda_iam_role, lambda_code_file_name)

In [14]:
# Pause to make sure agent is created
time.sleep(20)

# Configure and create 'Order' action group
order_agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=customer_support_agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': order_lambda_function['FunctionArn']
    },
    actionGroupName=order_agent_action_group_name,
    functionSchema={
        'functions': order_action_group_agent_functions
    },
    description=order_agent_action_group_description
)

In [ ]:
order_agent_action_group_response

# Provide permissions for Customer Support Agent to call the Action Group's Lambda functions
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [ ]:
# Allow agent to invoke permission on lambda functions for order action group
order_response = lambda_client.add_permission(
    FunctionName=order_lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{customer_support_agent_id}",
)
order_response

# Associate the knowledge base to the agent



In [ ]:
# Associate the knowledge base (you created in chapter 5) to the agent. Without this step, the agent will not be able to use the knowledge base.

response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=customer_support_agent_id,
    agentVersion='DRAFT',
    description='Use the provided knowledge base to find information related to product features, specifications, and FAQs about products, order and other policies.',
    knowledgeBaseId=customer_support_kb_id,
    knowledgeBaseState='ENABLED'
)

In [ ]:
# Let's Prepare the agent now.

response = bedrock_agent_client.prepare_agent(
    agentId=customer_support_agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

In [ ]:
# Prepare for Production Deployment - create alias
response = bedrock_agent_client.create_agent_alias(
    agentAliasName="test",
    agentId=customer_support_agent_id,
    description='Test alias',
)

alias_id = response["agentAlias"]["agentAliasId"]
print("The Agent alias is:",alias_id)
time.sleep(30)

In [3]:
def invokeAgent(query, session_id, enable_trace=False, session_state=dict()):
    end_session:bool = False
    
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=customer_support_agent_id,
        agentAliasId=alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session,
        sessionState=session_state
    )
    
    if enable_trace:
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [21]:
# Involve Agent to query Knowledge base

import uuid
session_id:str = str(uuid.uuid1())
query = "What are the features of the laptop?"
response = invokeAgent(query, session_id)
print(response)

The Laptop Pro 15 is a high-performance laptop suitable for professionals and gamers. It features a 15.6-inch 4K display, Intel i9 processor, 32GB RAM, 1TB SSD storage, and an NVIDIA RTX 3080 graphics card.


In [27]:
# Place order - Invoke Agent to execute functions from Action Group
session_id = str(uuid.uuid1())
query = "I want to order z12 wireless headphones and have them delivered to Rohit Singh at 2323 Solar Drive, Austin, Texas. Can you please use my default credit card for this order?"

response = invokeAgent(query, session_id)
print(response)

Thank you for your order. I have successfully placed an order for 1 unit of the z12 wireless headphones to be delivered to Rohit Singh at 2323 Solar Drive, Austin, Texas. The order ID is ORD40166 and the estimated delivery date is 2024-06-07. The payment was processed using your default credit card.


In [ ]:
# Retrieve order information - Invoke Agent to execute functions from Action Group
query = "I want to get the details for order ORD40166."  # Replace Order ID with your order ID from previous step
response = invokeAgent(query, session_id)
print(response)

In [ ]:
session_id:str = str(uuid.uuid1())
query = "Can you help cancel my order ORD40166?"
response = invokeAgent(query, session_id, enable_trace=True)
print(response)

# Clean up 

In [ ]:
clean_up_resources(
    table_name, order_lambda_function, order_lambda_function_name, order_agent_action_group_response, order_action_group_agent_functions, 
    customer_support_agent_id, customer_support_kb_id, alias_id
)

# Delete the agent roles and policies       

In [ ]:
delete_agent_roles_and_policies(agent_name)